In [8]:
import yfinance as yf
import pandas as pd
from joblib import Parallel, delayed

# Data Example #

In [2]:
msft = yf.Ticker("msft")

### History data ###

In [3]:
msft.history(period = "1y",
            interval = "1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-02-10 00:00:00-05:00,301.217530,306.250358,297.908540,299.572937,45386200,0.00,0.0
2022-02-11 00:00:00-05:00,300.375422,301.465217,291.488691,292.301086,39175600,0.00,0.0
2022-02-14 00:00:00-05:00,291.042882,294.005146,288.645364,292.261475,36359500,0.00,0.0
2022-02-15 00:00:00-05:00,297.225004,298.007648,294.262740,297.680725,27058300,0.00,0.0
2022-02-16 00:00:00-05:00,296.211422,298.693336,291.555350,297.333252,29982100,0.62,0.0
...,...,...,...,...,...,...,...
2023-02-03 00:00:00-05:00,259.540009,264.200012,257.100006,258.350006,29062300,0.00,0.0
2023-02-06 00:00:00-05:00,257.440002,258.299988,254.779999,256.769989,22518000,0.00,0.0
2023-02-07 00:00:00-05:00,260.529999,268.769989,260.079987,267.559998,50841400,0.00,0.0


### Options expirations ###

In [4]:
msft.options

('2023-02-10',
 '2023-02-17',
 '2023-02-24',
 '2023-03-03',
 '2023-03-10',
 '2023-03-17',
 '2023-03-24',
 '2023-04-21',
 '2023-05-19',
 '2023-06-16',
 '2023-07-21',
 '2023-08-18',
 '2023-09-15',
 '2024-01-19',
 '2024-06-21',
 '2025-01-17',
 '2025-06-20')

### Institutional_holders ###

In [5]:
msft.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",634238715,2022-09-29,0.0852,167198006951
1,Blackrock Inc.,518843676,2022-09-29,0.0697,136777567333
2,State Street Corporation,295509547,2022-09-29,0.0397,77902225337
3,"FMR, LLC",204088395,2022-09-29,0.0274,53801781693
4,Price (T.Rowe) Associates Inc,171346779,2022-09-29,0.0230,45170437043
5,"Geode Capital Management, LLC",139462260,2022-09-29,0.0187,36765040300
6,Morgan Stanley,104765804,2022-09-29,0.0141,27618360738
7,Capital World Investors,97528997,2022-09-29,0.0131,25710593712
8,JP Morgan Chase & Company,92745324,2022-09-29,0.0125,24449521860
9,Capital International Investors,84987239,2022-09-29,0.0114,22404335530


# Performance #

In [10]:
tickers_names = ["TSLA", "GOOGL", "GOOG", "F", "CS", "APP", "ALBKF", "SONO", "FNOXG", "FORM"]

In [27]:
def process_ticker_info(
        ticker_name: str,
        ticker_data: yf.Ticker,
        period: str,
        interval: str,
        use_ins_holders: bool = True) -> dict:
    result = {
        "ticker": ticker_name,
        "quotes": ticker_data.history(
            period=period,
            interval=interval,
            debug=False
        ),
    }

    if use_ins_holders:
        result["institutional_holders"] = ticker_data.institutional_holders

    return result

### With institutional holders info ###

In [30]:
%%timeit
result1 = [process_ticker_info(
    ticker_name, ticker_data, "1y", "1d"
)
    for ticker_name, ticker_data in yf.Tickers(
        tickers_names
    ).tickers.items()]

45.2 s ± 10.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Without institutional holders info

In [29]:
%%timeit
result2 = [process_ticker_info(
    ticker_name, ticker_data, "1y", "1d", use_ins_holders=False
)
    for ticker_name, ticker_data in yf.Tickers(
        tickers_names
    ).tickers.items()]

3.57 s ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
